In [2]:
#Generate Rossby wavenumber for circumglobal waves - model data

scenario = 'ssp126'
pathout ="/nfs/see-fs-01_teaching/ee21pnm/Project/output"
output_name = "/nfs/see-fs-01_teaching/ee21pnm/Spreadsheets/Rossby_"+scenario+".xlsx"
path = '/nfs/annie/earamay/Prash/'+scenario+'/'

lat_min = 35
lat_max =60
lon_min = -180 #-145
lon_max = 180 #-100

##IMPORT PACKAGES

import cartopy.feature as cfeature ## for lakes/borders et
countries_50m = cfeature.NaturalEarthFeature('cultural','admin_0_countries','50m',edgecolor='k',facecolor='none')
import cartopy.crs as ccrs
import numpy as np
from netCDF4 import Dataset  
import matplotlib.pyplot as plt
import math
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from shapely.geometry.polygon import LinearRing
import xarray as xr
import datetime as dt

import iris
from iris.experimental.equalise_cubes import equalise_attributes
from iris.util import unify_time_units
import iris.coord_categorisation
import iris.plot as iplt
import iris.quickplot as qplt

import pandas as pd

import glob



crs = ccrs.Orthographic(115,67)

##define constants
p0 = 1000   #base pressure level
rcp = 0.286 #gas constant divided by specific heat capacity
R_gas= 8.31 #ideal gas constant
G = 9.81 #gravity

#make function to convert all floats to integers in a matrix
def f1(x):
    return np.int(x)
f2 = np.vectorize(f1)


def suffix(d):
    return 'th' if 11<=d<=13 else {1:'st',2:'nd',3:'rd'}.get(d%10, 'th')

def custom_strftime(format, t):
    return t.strftime(format).replace('{S}', str(t.day) + suffix(t.day))

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w


def region_lat(input):
    return lat_min  <= input <= lat_max 

def region_long(input):
    return lon_min  <= input <= lon_max 


/apps/developers/compilers/canopy3/2.1.3/1/bit-64/Canopy/edm/envs/User/lib/python3.5/site-packages/matplotlib/ticker.py:1693: UserWarning: Steps argument should be a sequence of numbers
increasing from 1 to 10, inclusive. Behavior with
values outside this range is undefined, and will
raise a ValueError in future versions of mpl.
  warnings.warn('Steps argument should be a sequence of numbers\n'


In [4]:
year = '2050'
membername = 'r111i1p1f1'
vfile = glob.glob(path + "wind/va_day_EC-Earth3_"+scenario+"_" + membername + "_gr_" +str(year)+ "0101-" +str(year)+ "1231.nc")
fn = vfile[0]
cube1 = iris.load_cube(fn)
cube1

/apps/developers/libraries/python-libs/1.6/1/bit-64-python3-3.5.2/lib/iris/fileformats/cf.py:798: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'va'
  warnings.warn(message % (variable_name, nc_var_name))
/apps/developers/libraries/python-libs/1.6/1/bit-64-python3-3.5.2/lib/iris/fileformats/cf.py:1143: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


<iris 'Cube' of northward_wind / (m s-1) (time: 365; air_pressure: 3; latitude: 256; longitude: 512)>

In [6]:
pressure = cube1.coord('air_pressure')
print(pressure.points)

[ 85000.  50000.  25000.]


In [56]:
vfile = glob.glob('/nfs/annie/earamay/Prash/ssp126/wind/va_day_EC-Earth3_ssp126_r105i1p1f1_gr_20530101-20531231.nc')
cube1 = iris.load_cube(fn)

TranslationError: The file appears empty or incomplete: '/nfs/annie/earamay/Prash/ssp126/wind/va_day_EC-Earth3_ssp126_r105i1p1f1_gr_20520101-20521231.nc'

In [9]:
time = cube1.coord('time')
#print(time.points)
dt_time = [dt.date(1850, 1, 1) + dt.timedelta(days=int(t)) for t in time.points]
dates2.extend(dt_obj.strftime('%m/%d/%Y') for dt_obj in dt_time)
print(dt_time)

[ 73049.5  73050.5  73051.5  73052.5  73053.5  73054.5  73055.5  73056.5
  73057.5  73058.5  73059.5  73060.5  73061.5  73062.5  73063.5  73064.5
  73065.5  73066.5  73067.5  73068.5  73069.5  73070.5  73071.5  73072.5
  73073.5  73074.5  73075.5  73076.5  73077.5  73078.5  73079.5  73080.5
  73081.5  73082.5  73083.5  73084.5  73085.5  73086.5  73087.5  73088.5
  73089.5  73090.5  73091.5  73092.5  73093.5  73094.5  73095.5  73096.5
  73097.5  73098.5  73099.5  73100.5  73101.5  73102.5  73103.5  73104.5
  73105.5  73106.5  73107.5  73108.5  73109.5  73110.5  73111.5  73112.5
  73113.5  73114.5  73115.5  73116.5  73117.5  73118.5  73119.5  73120.5
  73121.5  73122.5  73123.5  73124.5  73125.5  73126.5  73127.5  73128.5
  73129.5  73130.5  73131.5  73132.5  73133.5  73134.5  73135.5  73136.5
  73137.5  73138.5  73139.5  73140.5  73141.5  73142.5  73143.5  73144.5
  73145.5  73146.5  73147.5  73148.5  73149.5  73150.5  73151.5  73152.5
  73153.5  73154.5  73155.5  73156.5  73157.5  7315

In [57]:
#weekly means


##IRIS - from Cathyrn Birch
import warnings
warnings.filterwarnings("ignore")

#create list to fill with all members
master_list = []

for membername in ['r111i1p1f1','r107i1p1f1','r110i1p1f1','r112i1p1f1','r109i1p1f1','r102i1p1f1','r105i1p1f1','r103i1p1f1','r106i1p1f1','r108i1p1f1','r101i1p1f1','r104i1p1f1']:
    #create empty lists
    dates =[]
    member = []

    #each wavenumber
    FFT1 =[]
    FFT2 =[]
    FFT3 =[]
    FFT4 =[]
    FFT5 =[]
    FFT6 =[]
    FFT7 =[]
    FFT8 =[]
    FFT9 =[]
    FFT10 =[]
    for year in np.arange(2015,2101,1):
        
        v250_yearly =[]
        
        #redirect for missing 2052 file
        if (year == 2052 and membername == 'r105i1p1f1' and scenario =='ssp126'): 
            vfile = glob.glob('/nfs/annie/ee21pnm/va_day_EC-Earth3_ssp126_r105i1p1f1_gr_20520101-20521231.nc')
        else:
            vfile = glob.glob(path + "wind/va_day_EC-Earth3_"+scenario+"_" + membername + "_gr_" +str(year)+ "0101-" +str(year)+ "1231.nc")
        fn = vfile[0]
        cube1 = iris.load_cube(fn)
        iris.coord_categorisation.add_month_number(cube1, 'time', name='month')
        
        #region constraint
        region_con = iris.Constraint(latitude = region_lat, longitude = region_long)


        # Access the longitude and latitude coordinates
        longitude = cube1.coord('longitude')
        latitude = cube1.coord('latitude')
        coords = ('latitude')
        pressure = cube1.coord('air_pressure')
        month = cube1.coord('month')

        #extract 250 hPa only
        pressure_con = iris.Constraint(air_pressure = float(25000))
        #extract June July only
        JuneJuly  = iris.Constraint(month=lambda m: m in (6,7))

        cube2 = cube1.extract(JuneJuly & region_con)
        cube3 = cube2.extract(iris.Constraint(air_pressure=25000))
        
        time = cube3.coord('time')

        for day in time.points:

            time_con = iris.Constraint(time = day)

            # Subset the cube
            time_subset = cube3.extract(time_con)

            result = time_subset.collapsed(coords, iris.analysis.MEAN)

            dt_time = dt.date(1850, 1, 1) + dt.timedelta(days=int(day))


            #append to lists
            dates.append(dt_time.strftime('%m/%d/%Y'))
            #member.append(membername)
            v250_yearly.append(result.data)

        #take yearly data to find running mean for each summer
        data = np.array(v250_yearly)
        df_yearly=pd.DataFrame(data=data,
        index=[i for i in range(data.shape[0])],
        columns=['long_'+str(i) for i in range(data.shape[1])])

        for i in range(data.shape[1]):
            df_yearly['weekly_long_'+str(i)] = df_yearly['long_'+str(i)].rolling(7).mean()

        #df_yearly

        #convert to array
        yearly_array = df_yearly.values
        #perform FFT
        for i in range(data.shape[0]):
            v250_FFT = yearly_array[i,360:720]
            fourier = np.fft.fft(v250_FFT)
            N = len(v250_FFT)
            #freq1 = np.arange(N/2)
            freq1 = np.arange(N)
            #fourier_1 = np.fft.fft(v_clim)
            #clim_fourier_2 = clim_fourier_1/N
            #clim_fourier_3 = clim_fourier_2[range(int(N/2))]
            FFT_Final = np.abs(fourier)
            #append to wavenumber lists
            FFT1.append(FFT_Final[1])
            FFT2.append(FFT_Final[2])
            FFT3.append(FFT_Final[3])
            FFT4.append(FFT_Final[4])
            FFT5.append(FFT_Final[5])
            FFT6.append(FFT_Final[6])
            FFT7.append(FFT_Final[7])
            FFT8.append(FFT_Final[8])
            FFT9.append(FFT_Final[9])
            FFT10.append(FFT_Final[10])

    member_array= np.repeat(membername,np.shape(dates))
    member_list = member_array.tolist()
    dates_list = [dates,member_list,FFT1,FFT2,FFT3,FFT4,FFT5,FFT6,FFT7,FFT8,FFT9,FFT10]
    #df['v250']=df['v250'].astype(float)
    print(membername + " done")
    #add to master list of all members
    master_list.extend(dates_list)
    
    
#df = pd.DataFrame(master_list).transpose()
#df.columns=['dates', 'member', 'wave-1','wave-2','wave-3','wave-4','wave-5','wave-6','wave-7','wave-8','wave-9','wave-10']
#df['dates'] = pd.to_datetime(df['dates'])

#df

r111i1p1f1 done
r107i1p1f1 done
r110i1p1f1 done
r112i1p1f1 done
r109i1p1f1 done
r102i1p1f1 done
r105i1p1f1 done
r103i1p1f1 done
r106i1p1f1 done
r108i1p1f1 done
r101i1p1f1 done
r104i1p1f1 done


ValueError: Length mismatch: Expected axis has 144 elements, new values have 12 elements

In [77]:
array= np.array(master_list)

array3 = np.reshape(array, (12, 62952), order='F')


df = pd.DataFrame(array3).transpose()
df.columns=['dates', 'member', 'wave-1','wave-2','wave-3','wave-4','wave-5','wave-6','wave-7','wave-8','wave-9','wave-10']
df['dates'] = pd.to_datetime(df['dates'])

df

,dates,member,wave-1,wave-2,wave-3,wave-4,wave-5,wave-6,wave-7,wave-8,wave-9,wave-10
0,2015-06-01,r111i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2015-06-01,r107i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2015-06-01,r110i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2015-06-01,r112i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2015-06-01,r109i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,2015-06-01,r102i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,2015-06-01,r105i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,2015-06-01,r103i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,2015-06-01,r106i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,2015-06-01,r108i1p1f1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [78]:
#save to excel

outname = "wavenumber_amplitude_"+scenario+".xlsx"
df.to_excel(outname)

In [10]:
df['wave-1'] =df['wave-1'].astype(float)
df['wave-2'] =df['wave-2'].astype(float)
df['wave-3'] =df['wave-3'].astype(float)
df['wave-4'] =df['wave-4'].astype(float)
df['wave-5'] =df['wave-5'].astype(float)
df['wave-6'] =df['wave-6'].astype(float)
df['wave-7'] =df['wave-7'].astype(float)
df['wave-8'] =df['wave-8'].astype(float)
df['wave-9'] =df['wave-9'].astype(float)
df['wave-10'] =df['wave-10'].astype(float)

In [11]:
df.dtypes

dates      datetime64[ns]
member             object
wave-1            float64
wave-2            float64
wave-3            float64
wave-4            float64
wave-5            float64
wave-6            float64
wave-7            float64
wave-8            float64
wave-9            float64
wave-10           float64
dtype: object

In [12]:
#test days
#df[df["dates"] =='2021-06-28'] # wave 4
#df[df["dates"] =='2020-07-05'] 

In [13]:
#find mean and stdv
std_1 = df['wave-1'].std()
mean_1 = df['wave-1'].mean()

std_2 = df['wave-2'].std()
mean_2 = df['wave-2'].mean()

std_3 = df['wave-3'].std()
mean_3 = df['wave-3'].mean()

std_4 = df['wave-4'].std()
mean_4 = df['wave-4'].mean()

std_5 = df['wave-5'].std()
mean_5 = df['wave-5'].mean()

std_6 = df['wave-6'].std()
mean_6 = df['wave-6'].mean()

std_7 = df['wave-7'].std()
mean_7 = df['wave-7'].mean()

std_8 = df['wave-8'].std()
mean_8 = df['wave-8'].mean()

std_9 = df['wave-9'].std()
mean_9 = df['wave-9'].mean()

std_10 = df['wave-10'].std()
mean_10 = df['wave-10'].mean()


In [14]:
#convert dataframe amplitudes to standard deviations above the mean

df2 = df.copy()

df2['wave-1'] = (df2['wave-1']-mean_1)/std_1
df2['wave-2'] = (df2['wave-2']-mean_2)/std_2
df2['wave-3'] = (df2['wave-3']-mean_3)/std_3
df2['wave-4'] = (df2['wave-4']-mean_4)/std_4
df2['wave-5'] = (df2['wave-5']-mean_5)/std_5
df2['wave-6'] = (df2['wave-6']-mean_6)/std_6
df2['wave-7'] = (df2['wave-7']-mean_7)/std_7
df2['wave-8'] = (df2['wave-8']-mean_8)/std_8
df2['wave-9'] = (df2['wave-9']-mean_9)/std_9
df2['wave-10'] = (df2['wave-10']-mean_10)/std_10

df2

,dates,member,wave-1,wave-2,wave-3,wave-4,wave-5,wave-6,wave-7,wave-8,wave-9,wave-10
0,2015-06-01,r111i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-06-01,r107i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-06-01,r110i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-01,r112i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-06-01,r109i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-06-01,r102i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-06-01,r105i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-06-01,r103i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-06-01,r106i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-06-01,r108i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
#test days
#df[df["dates"] =='2021-06-28'] # wave 4
#df2[df2["dates"] =='2021-06-28'] 

,dates,wave-1,wave-2,wave-3,wave-4,wave-5,wave-6,wave-7,wave-8,wave-9,wave-10
2589,2021-06-28,-0.123319,-0.129786,-0.146021,3.75988,0.869029,1.59015,-0.873161,-0.422446,0.702854,1.16538


In [61]:
#save df2 to excel, try plotting wave-5 and wave-7 against temps

In [15]:
outname = "wavenumber_stdv_"+scenario+".xlsx"
df2.to_excel(outname)